In [1]:
import scanpy as sc
import scvi
import scarches as sca
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Normal, Poisson
import torch.nn.functional as F
# from dalib.modules import domain_discriminator
# from scvi.nn._base_components import DecoderSCVI

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
Global seed set to 0
 captum (see https://github.com/pytorch/captum).


device(type='cuda')

In [2]:
ref_latent = sc.read_h5ad("./reference_latent_sample.h5ad")
# query_adata = sc.read_h5ad("/home/wyh/liver_atlas/data/Aizarani2019/Aizarani2019_plot_V2.h5ad")
query_adata = sc.read_h5ad("/home/wyh/liver_atlas/data/Ramachandran2019/Ramachandran2019_plot_V2.h5ad")
# ref_path = "./scvi_ref_model/"
ref_path = "./scvi_ref_model_sample/"

In [3]:
query_adata

AnnData object with n_obs × n_vars = 62210 × 33694
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Cell_ID', 'Sample_ID', 'Original_name_global', 'Original_name_sub', 'Patient_ID', 'Sample_status_original', 'Seq_tech', 'GEO_accession', 'Sample_status', 'Source', 'Gender', 'Age', 'Disease', 'Fibrotic_status', 'Author', 'Aetiology_of_Liver_Disease', 'level1', 'batch', 'leiden'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'leiden', 'level1_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [4]:
from scipy.sparse import csr_matrix
def feature_alignment(test_set, gene_list):
    test_set.X = test_set.layers['counts']
    
    # set test_set features as gene_list, zero-filling for missing features
    selected_set = set(gene_list)
    test_set_genes = set(test_set.var_names)
    common_set = selected_set & test_set_genes
    gene_extra = selected_set - common_set
    n_extra = len(gene_extra)
    if n_extra / len(gene_list) > 0.05:
        print("Warning: %d features not exist in testset." % len(gene_extra))

    if n_extra > 0:  # fill zeros for missing features
        new_mtx = csr_matrix(test_set.X, shape=(test_set.n_obs, test_set.n_vars + n_extra))
        test_adata = sc.AnnData(new_mtx)
        test_adata.obs = test_set.obs
        test_adata.layers['counts'] = test_adata.X
        test_adata.obs_names = test_set.obs_names
        test_adata.var_names = list(test_set.var_names) + list(gene_extra)
        # test_adata.obs = test_set.obs
        return test_adata[:, gene_list]
    else:
        return test_set[:, gene_list]
    
gene_selected = pd.read_csv(ref_path + "var_names.csv", header=None)[0].tolist()
query_adata = feature_alignment(query_adata, gene_selected)
query_adata.obs['sample'] = query_adata.obs['batch']

scvi_model = sca.models.SCVI.load_query_data(
    query_adata,
    ref_path,
    freeze_dropout = True,
)

scvi_model.train(max_epochs=0, plan_kwargs=dict(weight_decay=0.0))
# scvi.data.view_anndata_setup(scvi_model.adata)

/tmp/ipykernel_2499457/4234022668.py:16: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  test_adata = sc.AnnData(new_mtx)
/tmp/ipykernel_2499457/4234022668.py:28: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  query_adata.obs['sample'] = query_adata.obs['batch']


INFO     Using data from adata.layers["counts"]                                              


/home/wyh/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/scvi/model/base/_archesmixin.py:95: UserWarning: Query integration should be performed using models trained with version >= 0.8
  warnings.warn(
/home/wyh/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function transfer_anndata_setup is deprecated; This method will be removed in 0.15.0. Please avoid building any new dependencies on it.
  warnings.warn(msg, category=FutureWarning)
INFO:scvi.data._anndata:Using data from adata.layers["counts"]


INFO     Registered keys:['X', 'batch_indices', 'labels']                                    


INFO:scvi.data._anndata:Registered keys:['X', 'batch_indices', 'labels']


INFO     Successfully registered anndata object containing 62210 cells, 2000 vars, 39        
         batches, 10 labels, and 0 proteins. Also registered 0 extra categorical covariates  
         and 0 extra continuous covariates.                                                  


INFO:scvi.data._anndata:Successfully registered anndata object containing 62210 cells, 2000 vars, 39 batches, 10 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra continuous covariates.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Training: 0it [00:00, ?it/s]

In [5]:
scvi.data.view_anndata_setup(scvi_model.adata)

/home/wyh/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function view_anndata_setup is deprecated; This method will be removed in 0.15.0. Please avoid building any new dependencies on it.
  warnings.warn(msg, category=FutureWarning)


Anndata setup with scvi-tools version 0.14.6.

              Data Summary              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃             Data             ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│            Cells             │ 62210 │
│             Vars             │ 2000  │
│            Labels            │  10   │
│           Batches            │  39   │
│           Proteins           │   0   │
│ Extra Categorical Covariates │   0   │
│ Extra Continuous Covariates  │   0   │
└──────────────────────────────┴───────┘

             SCVI Data Registry              
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃     Data      ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       X       │  adata.layers['counts']   │
│ batch_indices │ adata.obs['_scvi_batch']  │
│    labels     │ adata.obs['_scvi_labels'] │
└───────────────┴───────────────────────────┘

                        Label Categories                        
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃   Source Location   ┃    Categories    ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['level1'] │      B cell      │          0          │
│                     │  Cholangiocyte   │          1          │
│                     │ Endothelial cell │          2          │
│                     │    Hepatocyte    │          3          │
│                     │ Mesenchymal cell │          4          │
│                     │   Myeloid cell   │          5          │
│                     │    NK/T cell     │          6          │
│                     │  Plasma B cell   │          7          │
│                     │     Cycling      │          8          │
│                     │     Unknown      │          9          │
└─────────────────────┴──────────────────┴─────────────────────┘

                        Batch Categories                        
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃   Source Location   ┃    Categories    ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['sample'] │       ABU8       │          0          │
│                     │      CISE06      │          1          │
│                     │      CISE07      │          2          │
│                     │      CISE08      │          3          │
│                     │      CISE09      │          4          │
│                     │       CS31       │          5          │
│                     │       CS32       │          6          │
│                     │       CS33       │          7          │
│                     │       CS34       │          8          │
│                     │       CS37       │          9          │
│                     │       CS38       │         10          │
│                     │       CS41       │         11          │
│                     │       CS42       │         12          │
│                     │       CS43       │         13          │
│                     │       CS44       │         14          │
│                     │       CS46       │         15          │
│                     │       CS71       │         16          │
│                     │       CS73       │         17          │
│                     │       CS81       │         18          │
│                     │       CS83       │         19          │
│                     │       CS85       │         20          │
│                     │       CS87       │         21          │
│                     │      CS101       │         22          │
│                     │      CS108       │         23          │
│                     │      CS109       │         24          │
│                     │      CS110       │         25          │
│                     │      CS111       │         26          │
│                     │      CS112       │         27          │
│                     │      CS126       │         28          │
│                     │      CS127       │         29          │
│                     │      CS161       │         30          │
│                     │      CS162       │         31          │
│                     │      CS164       │         32          │
│                     │      CS166       │         33          │
│                     │      CS167       │         34          │
│                     │      CS169       │         35          │
│                     │      CS170       │         36          │
│                     │      CS171       │         37          │
│                     │ Ramachandran2019 │         38          │
└─────────────────────┴──────────────────┴─────────────────────┘

In [6]:
batch_id = scvi_model.adata.obs['_scvi_batch'][0]

In [7]:
latent_origin = sc.AnnData(scvi_model.get_latent_representation())
latent_origin.obs['cell_type'] = scvi_model.adata.obs['level1'].tolist()
latent_origin.obs['batch'] = scvi_model.adata.obs['batch'].tolist()
latent_origin.write("ramachandran_latent_origin.h5ad")

In [8]:
class TransferMappingModule(nn.Module):
    def __init__(
        self, 
        n_latent=10, 
        n_gene=2000, 
        n_hidden=[64, 32],
        dropout_rate=0.1,
        extra_decoder_kwargs=None
    ):
        
        
        super(TransferMappingModule, self).__init__()
        
#         # decoder (froze param)
#         self.decoder = Decoder(
#             n_latent=n_latent, 
#             n_out=n_gene, 
#         )

        # transfer
        self.fc11 = nn.Linear(n_latent, n_hidden[0])
        self.fc12 = nn.Linear(n_hidden[0], n_hidden[1])      
        self.fc2 = nn.Linear(n_hidden[1], n_latent)
        self.dropout = nn.Dropout(p=dropout_rate)
        
    def query_transfer(self, z):
        h = F.relu(self.fc11(z))
        h = self.dropout(h)
        h = F.relu(self.fc12(h))
        h = self.dropout(h)
        return self.fc2(h)
    
#     def generative(self, z):
#         categorical_input = ()
#         size_factor = None
# #         px_scale, px_r, px_rate, px_dropout = self.decoder(
# #                 self.dispersion,
# #                 self.query_transfer(z),
# #                 size_factor,
# #                 *categorical_input,
# #                 y,
# #             )
# #         px = NegativeBinomial(mu=px_rate, theta=px_r, scale=px_scale)
#         px_scale, px_dropout = self.decoder(z)
#         poisson = Poisson(torch.exp(library) * px_scale + self.eps)
#         decoder_poisson = Poisson(px_scale + self.eps)
#         return px
    
    def forward(self, z):
        return self.query_transfer(z)

# 定义discriminator类
class DomainDiscriminator(nn.Module):
    def __init__(self, input_dim=30, hidden_dim=64, dropout_rate=0.1):
        super(DomainDiscriminator, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, dropout_rate=0.1):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return self.sigmoid(x)

In [9]:
from torch.utils.data import Dataset, DataLoader

# 自定义dataset类
class MyDataset(Dataset):
    def __init__(self, adata, X_key):
        self.data = adata
        if X_key == "X":
            self.X = adata.X
        else:
            self.X = adata.layers[X_key]
        
    def __len__(self):
        return self.data.n_obs
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx,:].toarray())
    

In [10]:
def discriminator_loss(discriminator, ref_latent, query_latent, mode):
    if mode == "trainD":
        ref_labels = torch.zeros(ref_latent.shape[0], 1)
        query_labels = torch.ones(query_latent.shape[0], 1)
        labels = torch.cat((ref_labels, query_labels), dim=0).to(device)
        
    elif mode == "trainM":
        labels = torch.zeros(ref_latent.shape[0] + query_latent.shape[0], 1).to(device)
    else:
        print("No such mode for discriminator loss.")

    # 将源领域和目标领域数据合并并输入到域鉴别器中
    data = torch.cat((ref_latent, query_latent), dim=0).to(device)
    predictions = discriminator(data)

    # 计算域鉴别器的损失
    criterion = nn.BCELoss()
    loss = criterion(predictions, labels)
#     print(mode)
#     print(predictions[-10:])
#     print(labels[-10:])
#     print(loss)
    return loss #* 1e3

def reconstruction_loss(scvi_model, transfer_mapping_module, X, batch_id):
    n_sample = X.shape[0]
    batch_index = torch.full((n_sample, 1), batch_id)
    inference_output = scvi_model.module.inference(
        X, 
        batch_index=batch_index
    )
    z = inference_output['z']
    transferred_z = transfer_mapping_module(z)
    generative_output = scvi_model.module.generative(
        z=transferred_z,
        library=inference_output['library'], 
        batch_index=batch_index
    )    
    reconst_loss = scvi_model.module.get_reconstruction_loss(
        X, 
        generative_output['px_rate'], 
        generative_output['px_r'], 
        generative_output['px_dropout']
    )
#     loss = -generative_x.log_prob(x).sum(-1)
    return torch.mean(reconst_loss)


In [11]:
import scanpy as sc
dataset = MyDataset(query_adata, "counts")

# hyper-parameters
learning_rate = 0.01
batch_size = 256
num_workers = 4
shuffle = True
num_epoch = 250

# Instantiate trainloader
train_loader = DataLoader(dataset, 
                        batch_size=batch_size, 
                        num_workers=num_workers, 
                        shuffle=shuffle)
# Instantiate mapping module
mapping_module = TransferMappingModule(n_latent=30, n_gene=2000,  
                                       dropout_rate=0.1).to(device)
# Instantiate discriminator
discriminator = DomainDiscriminator(input_dim=30, 
                                    dropout_rate=0.1).to(device)

# criterion = nn.BCELoss()
optimizerD = optim.Adam(discriminator.parameters(), lr=learning_rate)
optimizerM = optim.Adam(mapping_module.parameters(), lr=learning_rate)

from torch.optim.lr_scheduler import StepLR
schedulerD = StepLR(optimizerD, step_size=50, gamma=0.5)
schedulerM = StepLR(optimizerM, step_size=50, gamma=0.5)

In [12]:
reconst_loss_rate = 0.001
num_batches = len(train_loader)

mapping_module.train()
discriminator.train()

z_record = {}
for epoch in range(num_epoch):
    # ref_latent to GPU
    ref_latent_subset = sc.pp.subsample(ref_latent, n_obs = 10000, 
                                        random_state = epoch, copy=True)
    ref_latent_tensor = torch.tensor(ref_latent_subset.X).to(device)
    
    for i, x in enumerate(train_loader):
        query_x = x.to(device)
        # encode and transfer
        n_sample = query_x.shape[0]
        batch_index = torch.full((n_sample, 1), batch_id)
        inference_output = scvi_model.module.inference(
            query_x, 
            batch_index=batch_index
        )
        query_latent = torch.squeeze(inference_output['z'])
        query_latent_transferred = mapping_module(query_latent)
        
        ##############################
        #   Training discriminator   #
        ##############################
        # forward
        lossD = discriminator_loss(
            discriminator, 
            ref_latent_tensor, 
            query_latent_transferred,
            "trainD"
        )
        # backward
        optimizerD.zero_grad()
        lossD.backward()
#         lossD.backward(retain_graph=True)
        optimizerD.step()         
        
#         ##########################
#         #   Training generator   #
#         ##########################
        inference_output = scvi_model.module.inference(
            query_x, 
            batch_index=batch_index
        )
        query_latent = torch.squeeze(inference_output['z'])
        query_latent_transferred = mapping_module(query_latent)
        
        # forward
        discrim_loss = discriminator_loss(
            discriminator, 
            ref_latent_tensor, 
            query_latent_transferred,
            "trainM"
        )
        reconst_loss = reconstruction_loss(scvi_model, mapping_module, 
                                           torch.squeeze(query_x), batch_id)
        lossM = reconst_loss_rate * reconst_loss + discrim_loss
        
        # backward
        optimizerM.zero_grad()
        lossM.backward()
        optimizerM.step()
        
        if i == 0:
            z_mean = torch.tensor(scvi_model.get_latent_representation()).to(device)
            z_mean_transferred = mapping_module(z_mean)
            z_record[epoch] = z_mean_transferred
#             break     

        if i%40 == 0:
#             print(lossD.item())
            print('Epoch [{}/{}], step [{}/{}], D_loss: {:.4f}, M_loss: {:.4f}, M_d_loss: {:.4f}, M_r_loss: {:.4f}'.format(
                epoch+1, num_epoch, i+1, num_batches, 
                lossD.item(), lossM.item(), 
                discrim_loss.item(), reconst_loss.item()))
    schedulerD.step()
    schedulerM.step()



/home/wyh/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:97: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/wyh/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/cuda/jit_utils.cpp:860.)
  + torch.lgamma(x + theta)


Epoch [1/250], step [1/244], D_loss: 0.7648, M_loss: 2.0078, M_d_loss: 0.6395, M_r_loss: 1368.3115
Epoch [1/250], step [41/244], D_loss: 0.0392, M_loss: 0.6529, M_d_loss: 0.0307, M_r_loss: 622.2471
Epoch [1/250], step [81/244], D_loss: 0.1358, M_loss: 0.6260, M_d_loss: 0.0416, M_r_loss: 584.4247
Epoch [1/250], step [121/244], D_loss: 0.0651, M_loss: 0.6175, M_d_loss: 0.0235, M_r_loss: 594.0700
Epoch [1/250], step [161/244], D_loss: 0.1304, M_loss: 0.6688, M_d_loss: 0.0622, M_r_loss: 606.6487
Epoch [1/250], step [201/244], D_loss: 0.0487, M_loss: 0.6529, M_d_loss: 0.0381, M_r_loss: 614.7516
Epoch [1/250], step [241/244], D_loss: 0.0540, M_loss: 0.5459, M_d_loss: 0.0325, M_r_loss: 513.4702
Epoch [2/250], step [1/244], D_loss: 0.0509, M_loss: 0.5717, M_d_loss: 0.0300, M_r_loss: 541.7722
Epoch [2/250], step [41/244], D_loss: 0.0700, M_loss: 0.5859, M_d_loss: 0.0354, M_r_loss: 550.4769
Epoch [2/250], step [81/244], D_loss: 0.0535, M_loss: 0.5690, M_d_loss: 0.0249, M_r_loss: 544.0588
Epoch [

Epoch [12/250], step [241/244], D_loss: 0.0743, M_loss: 0.5985, M_d_loss: 0.0404, M_r_loss: 558.1320
Epoch [13/250], step [1/244], D_loss: 0.0894, M_loss: 0.6307, M_d_loss: 0.0418, M_r_loss: 588.8945
Epoch [13/250], step [41/244], D_loss: 0.0795, M_loss: 0.6280, M_d_loss: 0.0414, M_r_loss: 586.5339
Epoch [13/250], step [81/244], D_loss: 0.0670, M_loss: 0.6467, M_d_loss: 0.0414, M_r_loss: 605.3606
Epoch [13/250], step [121/244], D_loss: 0.0482, M_loss: 0.6118, M_d_loss: 0.0286, M_r_loss: 583.1709
Epoch [13/250], step [161/244], D_loss: 0.0541, M_loss: 0.6119, M_d_loss: 0.0327, M_r_loss: 579.2499
Epoch [13/250], step [201/244], D_loss: 0.0694, M_loss: 0.6079, M_d_loss: 0.0467, M_r_loss: 561.2422
Epoch [13/250], step [241/244], D_loss: 0.0665, M_loss: 0.6605, M_d_loss: 0.0598, M_r_loss: 600.7540
Epoch [14/250], step [1/244], D_loss: 0.0903, M_loss: 0.6248, M_d_loss: 0.0575, M_r_loss: 567.2628
Epoch [14/250], step [41/244], D_loss: 0.0663, M_loss: 0.5958, M_d_loss: 0.0360, M_r_loss: 559.81

Epoch [24/250], step [161/244], D_loss: 0.0684, M_loss: 0.6562, M_d_loss: 0.0423, M_r_loss: 613.8911
Epoch [24/250], step [201/244], D_loss: 0.0553, M_loss: 0.5835, M_d_loss: 0.0377, M_r_loss: 545.7734
Epoch [24/250], step [241/244], D_loss: 0.0503, M_loss: 0.5758, M_d_loss: 0.0416, M_r_loss: 534.1534
Epoch [25/250], step [1/244], D_loss: 0.0536, M_loss: 0.5602, M_d_loss: 0.0384, M_r_loss: 521.7930
Epoch [25/250], step [41/244], D_loss: 0.0735, M_loss: 0.5882, M_d_loss: 0.0361, M_r_loss: 552.0939
Epoch [25/250], step [81/244], D_loss: 0.0596, M_loss: 0.5977, M_d_loss: 0.0434, M_r_loss: 554.3300
Epoch [25/250], step [121/244], D_loss: 0.0758, M_loss: 0.6087, M_d_loss: 0.0368, M_r_loss: 571.8636
Epoch [25/250], step [161/244], D_loss: 0.0919, M_loss: 0.5948, M_d_loss: 0.0498, M_r_loss: 545.0219
Epoch [25/250], step [201/244], D_loss: 0.0692, M_loss: 0.5828, M_d_loss: 0.0430, M_r_loss: 539.8155
Epoch [25/250], step [241/244], D_loss: 0.0695, M_loss: 0.5743, M_d_loss: 0.0442, M_r_loss: 530

Epoch [36/250], step [81/244], D_loss: 0.0425, M_loss: 0.5526, M_d_loss: 0.0391, M_r_loss: 513.4408
Epoch [36/250], step [121/244], D_loss: 0.0636, M_loss: 0.5799, M_d_loss: 0.0425, M_r_loss: 537.3907
Epoch [36/250], step [161/244], D_loss: 0.0942, M_loss: 0.6299, M_d_loss: 0.0479, M_r_loss: 582.0370
Epoch [36/250], step [201/244], D_loss: 0.0657, M_loss: 0.6607, M_d_loss: 0.0391, M_r_loss: 621.6284
Epoch [36/250], step [241/244], D_loss: 0.0524, M_loss: 0.5926, M_d_loss: 0.0364, M_r_loss: 556.1967
Epoch [37/250], step [1/244], D_loss: 0.0623, M_loss: 0.6153, M_d_loss: 0.0397, M_r_loss: 575.6472
Epoch [37/250], step [41/244], D_loss: 0.0484, M_loss: 0.5556, M_d_loss: 0.0410, M_r_loss: 514.5375
Epoch [37/250], step [81/244], D_loss: 0.0503, M_loss: 0.6000, M_d_loss: 0.0415, M_r_loss: 558.4802
Epoch [37/250], step [121/244], D_loss: 0.0409, M_loss: 0.5766, M_d_loss: 0.0418, M_r_loss: 534.7502
Epoch [37/250], step [161/244], D_loss: 0.0611, M_loss: 0.5742, M_d_loss: 0.0363, M_r_loss: 537.

...

Epoch [187/250], step [161/244], D_loss: 0.0595, M_loss: 0.5625, M_d_loss: 0.0438, M_r_loss: 518.6321
Epoch [187/250], step [201/244], D_loss: 0.0581, M_loss: 0.5660, M_d_loss: 0.0410, M_r_loss: 525.0203
Epoch [187/250], step [241/244], D_loss: 0.0593, M_loss: 0.5767, M_d_loss: 0.0385, M_r_loss: 538.1666
Epoch [188/250], step [1/244], D_loss: 0.0591, M_loss: 0.6275, M_d_loss: 0.0399, M_r_loss: 587.6102
Epoch [188/250], step [41/244], D_loss: 0.0498, M_loss: 0.5450, M_d_loss: 0.0390, M_r_loss: 505.9753
Epoch [188/250], step [81/244], D_loss: 0.0565, M_loss: 0.5845, M_d_loss: 0.0410, M_r_loss: 543.5083
Epoch [188/250], step [121/244], D_loss: 0.0775, M_loss: 0.5882, M_d_loss: 0.0432, M_r_loss: 545.0168
Epoch [188/250], step [161/244], D_loss: 0.0615, M_loss: 0.5827, M_d_loss: 0.0488, M_r_loss: 533.8512
Epoch [188/250], step [201/244], D_loss: 0.0675, M_loss: 0.5964, M_d_loss: 0.0417, M_r_loss: 554.6987
Epoch [188/250], step [241/244], D_loss: 0.0534, M_loss: 0.6136, M_d_loss: 0.0386, M_r

Epoch [199/250], step [41/244], D_loss: 0.0525, M_loss: 0.5763, M_d_loss: 0.0442, M_r_loss: 532.1790
Epoch [199/250], step [81/244], D_loss: 0.0528, M_loss: 0.6074, M_d_loss: 0.0407, M_r_loss: 566.7642
Epoch [199/250], step [121/244], D_loss: 0.0561, M_loss: 0.5964, M_d_loss: 0.0391, M_r_loss: 557.3458
Epoch [199/250], step [161/244], D_loss: 0.0489, M_loss: 0.5655, M_d_loss: 0.0372, M_r_loss: 528.3440
Epoch [199/250], step [201/244], D_loss: 0.0533, M_loss: 0.5864, M_d_loss: 0.0413, M_r_loss: 545.1699
Epoch [199/250], step [241/244], D_loss: 0.0606, M_loss: 0.5796, M_d_loss: 0.0389, M_r_loss: 540.6922
Epoch [200/250], step [1/244], D_loss: 0.0634, M_loss: 0.5847, M_d_loss: 0.0399, M_r_loss: 544.7720
Epoch [200/250], step [41/244], D_loss: 0.0565, M_loss: 0.5936, M_d_loss: 0.0386, M_r_loss: 555.0253
Epoch [200/250], step [81/244], D_loss: 0.0474, M_loss: 0.6337, M_d_loss: 0.0432, M_r_loss: 590.5939
Epoch [200/250], step [121/244], D_loss: 0.0526, M_loss: 0.5627, M_d_loss: 0.0375, M_r_l

Epoch [210/250], step [201/244], D_loss: 0.0609, M_loss: 0.5580, M_d_loss: 0.0375, M_r_loss: 520.5202
Epoch [210/250], step [241/244], D_loss: 0.0701, M_loss: 0.5915, M_d_loss: 0.0392, M_r_loss: 552.3030
Epoch [211/250], step [1/244], D_loss: 0.0659, M_loss: 0.5837, M_d_loss: 0.0418, M_r_loss: 541.9391
Epoch [211/250], step [41/244], D_loss: 0.0567, M_loss: 0.6435, M_d_loss: 0.0432, M_r_loss: 600.2681
Epoch [211/250], step [81/244], D_loss: 0.0583, M_loss: 0.5905, M_d_loss: 0.0388, M_r_loss: 551.7493
Epoch [211/250], step [121/244], D_loss: 0.0559, M_loss: 0.5483, M_d_loss: 0.0384, M_r_loss: 509.8801
Epoch [211/250], step [161/244], D_loss: 0.0519, M_loss: 0.5830, M_d_loss: 0.0427, M_r_loss: 540.2804
Epoch [211/250], step [201/244], D_loss: 0.0575, M_loss: 0.5687, M_d_loss: 0.0419, M_r_loss: 526.8436
Epoch [211/250], step [241/244], D_loss: 0.0575, M_loss: 0.5830, M_d_loss: 0.0381, M_r_loss: 544.8868
Epoch [212/250], step [1/244], D_loss: 0.0529, M_loss: 0.6403, M_d_loss: 0.0446, M_r_l

Epoch [222/250], step [81/244], D_loss: 0.0490, M_loss: 0.5852, M_d_loss: 0.0434, M_r_loss: 541.7753
Epoch [222/250], step [121/244], D_loss: 0.0504, M_loss: 0.5855, M_d_loss: 0.0438, M_r_loss: 541.7225
Epoch [222/250], step [161/244], D_loss: 0.0547, M_loss: 0.5872, M_d_loss: 0.0403, M_r_loss: 546.9265
Epoch [222/250], step [201/244], D_loss: 0.0536, M_loss: 0.5889, M_d_loss: 0.0398, M_r_loss: 549.1802
Epoch [222/250], step [241/244], D_loss: 0.0635, M_loss: 0.5919, M_d_loss: 0.0375, M_r_loss: 554.3319
Epoch [223/250], step [1/244], D_loss: 0.0667, M_loss: 0.5524, M_d_loss: 0.0411, M_r_loss: 511.2664
Epoch [223/250], step [41/244], D_loss: 0.0653, M_loss: 0.5792, M_d_loss: 0.0401, M_r_loss: 539.0455
Epoch [223/250], step [81/244], D_loss: 0.0600, M_loss: 0.5939, M_d_loss: 0.0429, M_r_loss: 550.9951
Epoch [223/250], step [121/244], D_loss: 0.0577, M_loss: 0.5908, M_d_loss: 0.0425, M_r_loss: 548.3073
Epoch [223/250], step [161/244], D_loss: 0.0493, M_loss: 0.5964, M_d_loss: 0.0451, M_r_

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 23.70 GiB total capacity; 15.25 GiB already allocated; 6.00 MiB free; 16.37 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
save_path = './saves'
ad = sc.AnnData(scvi_model.get_latent_representation())
ad.obs["cell_type"] = query_adata.obs["level1"].tolist()
ad.obs["batch"] = query_adata.obs["batch"].tolist()
ad.raw = ad
ad.write(save_path + "/ad_sample_ramachandran_origin.h5ad")

In [13]:
schedulerM.get_lr()

/home/wyh/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


[0.000625]

In [14]:
# import os
model_path = './model'
# os.makedirs(model_path, exist_ok=True)
torch.save(discriminator.state_dict(), model_path + "/sample_ramachandran_230.pth")
torch.save(mapping_module.state_dict(), model_path + "/sample_ramachandran_230.pth")

In [15]:
save_path = './saves'
ad = sc.AnnData(z_record[0].detach().cpu().numpy())
ad.obs["cell_type"] = query_adata.obs["level1"].tolist()
ad.obs["batch"] = query_adata.obs["batch"].tolist()
ad.raw = ad
ad.write(save_path + "/ad_sample_ramachandran_epo00.h5ad")

save_path = './saves'
ad = sc.AnnData(z_record[99].detach().cpu().numpy())
ad.obs["cell_type"] = query_adata.obs["level1"].tolist()
ad.obs["batch"] = query_adata.obs["batch"].tolist()
ad.raw = ad
ad.write(save_path + "/ad_sample_ramachandran_epo99.h5ad")

# ad = sc.AnnData(z_record[249].detach().cpu().numpy())
ad = sc.AnnData(z_record[229].detach().cpu().numpy())
ad.obs["cell_type"] = query_adata.obs["level1"].tolist()
ad.obs["batch"] = query_adata.obs["batch"].tolist()
ad.raw = ad
ad.write(save_path + "/ad_sample_ramachandran.h5ad")

In [17]:
save_path = './saves'
# os.makedirs(save_path, exist_ok=True)

reference_latent1 = sc.AnnData(scvi_model.get_latent_representation())
reference_latent1.obs["cell_type"] = query_adata.obs["level1"].tolist()
reference_latent1.obs["batch"] = query_adata.obs["batch"].tolist()
reference_latent1.write(save_path + "/latent_origin2.h5ad")

z_mean = torch.tensor(scvi_model.get_latent_representation()).to(device)
z_mean_transferred = mapping_module(z_mean)
reference_latent = sc.AnnData(z_mean_transferred.detach().cpu().numpy())
reference_latent.obs["cell_type"] = query_adata.obs["level1"].tolist()
reference_latent.obs["batch"] = query_adata.obs["batch"].tolist()
reference_latent.write(save_path + "/latent_transferred2.h5ad")

/home/wyh/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/scvi/model/base/_base_model.py:150: UserWarning: Make sure the registered X field in anndata contains unnormalized count data.
  warnings.warn(
/home/wyh/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/scvi/model/base/_base_model.py:150: UserWarning: Make sure the registered X field in anndata contains unnormalized count data.
  warnings.warn(


In [ ]:
import scanpy as sc
save_path = './saves'
query_latent_origin = sc.read_h5ad(save_path + "/latent_origin2.h5ad")
query_latent_transferred = sc.read_h5ad(save_path + "/latent_transferred2.h5ad")

In [ ]:
ref_latent = sc.read_h5ad("./reference_latent.h5ad")
sc.pp.subsample(ref_latent, n_obs = 10000)
merged_adata = sc.AnnData.concatenate(ref_latent, query_latent_transferred,
                                      batch_key="ref_query")
merged_adata

In [ ]:
sc.pp.neighbors(merged_adata, n_neighbors=4)
sc.tl.umap(merged_adata)
sc.pl.umap(merged_adata,
           color=['cell_type', "ref_query"],
           frameon=False,
           wspace=0.6,
           )

In [ ]:
merged_adata0 = sc.AnnData.concatenate(ref_latent, query_latent_origin,
                                      batch_key = "ref_query")

sc.pp.neighbors(merged_adata0, n_neighbors=4)
sc.tl.umap(merged_adata0)
sc.pl.umap(merged_adata0,
           color=['cell_type', "ref_query"],
           frameon=False,
           wspace=0.6,
           )